# 1. Setting up the environment

# 1.1 Checking if the notebook was launched in the correct environment

In [ ]:
#prints the available list of environments with asterisk (*) next to the current environment
#need to relaunch jupyter if launched in incorrect environment
!conda env list

## 1.2 Installing the required packages in the environment

In [ ]:
#prints the list of installed packages in the environment
#useful for checking versions and confirming if the packages have successfully installed
!conda list

In [ ]:
#installing the required packages
import sys
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} opencv
!conda install --yes --prefix {sys.prefix} scikit-image
!conda install --yes --prefix {sys.prefix} scikit-learn
!conda install -c conda-forge --yes --prefix {sys.prefix} python-sounddevice
!conda install -c conda-forge  --yes --prefix {sys.prefix} pysoundfile

## 1.3 Importing the installed packages in the environment

In [1]:
#importing the packages in the environment for use

import os #module allowing interaction with the file system
import time #module to handle time-related tasks such as model training time etc.

import numpy as np #module for added support to large, multi-dimensional arrays
import pandas as pd #module for data manipulation and analysis 

import pickle #module for exporting the model
import cv2 #real-time optimized Computer Vision library

from skimage.feature import hog #module for feature descriptor method - HOG (Histogram of Oriented Gradients)

from threading import Thread #module allowing for multiple threads in the program
import sounddevice as sd #module to read/access sound files
import soundfile as sf #module to play sound files

# 2. Defining functions for miscellaneous operations

## 2.1 Function to detect eye status

In [37]:
def detect_drowsy(gray, frame, tempstat):
    
    #eye probabilities
    eye_prob1 = 'N/A'
    eye_prob2 = 'N/A'
    
    #initialising face cascade for face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    #checking if no face available
    if len(faces) < 1:
        tempstat = 'NO FACE DETECTED'
        
    else:
        #for each face detected
        for (x,y,w,h) in faces:
            roi_color=frame[y:y+h, x:x+w]
            roi_gray=gray[y:y+h, x:x+w]
            
            #show a blue box around it
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 1)
            
            #initialising eye cascade for eyes detection
            eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10, minSize=(15,15), flags=cv2.CASCADE_SCALE_IMAGE)
            
            #checking if no eyes available
            if len(eyes) < 1:
                tempstat = 'NO EYES DETECTED'
            
            #when there are 2 eyes
            elif len(eyes) == 2:
                eye1 = eyes[0]
                eye2 = eyes[1]
                
                #getting coordinates for each eye
                ex1, ey1, ew1, eh1 = eye1
                ex2, ey2, ew2, eh2 = eye2

                #show a green box around it
                cv2.rectangle(roi_color, (ex1,ey1), (ex1+ew1, ey1+eh1), (0, 255, 0), 1)
                cv2.rectangle(roi_color, (ex2,ey2), (ex2+ew2, ey2+eh2), (0, 255, 0), 1)    
                
                #extract that image
                eye_image1 = roi_gray[ey1:ey1+eh1, ex1:ex1+ew1]
                eye_image2 = roi_gray[ey2:ey2+eh2, ex2:ex2+ew2]
                
                #resize that image
                eye_image1 = cv2.resize(eye_image1, (80,80))
                eye_image2 = cv2.resize(eye_image2, (80,80))
                
                #extract features from image
                eye_hog1 = get_hog(eye_image1)
                eye_hog2 = get_hog(eye_image2)
                
                #predict eye state on those features
                eye_pred1 = svm_hog_model.predict([eye_hog1])
                eye_pred2 = svm_hog_model.predict([eye_hog2])
                
                #if both eyes are closed, show status 'EYES CLOSED'
                if eye_pred1 == eye_pred2 == 0:
                    tempstat = 'EYES CLOSED'
                
                #if both eyes are open, show status 'EYES OPEN'
                elif eye_pred1 == eye_pred2 == 1:
                    tempstat = 'EYES OPEN'
                #else show 'EYES OPEN'
                else:
                    tempstat = 'PARTIAL DETECTION'
                    
                #predict probabilities for eye states on those features
                eye_prob1 = "%.2f" % (round((svm_hog_model.predict_proba([eye_hog1])[:,1])[0], 4) * 100)
                eye_prob2 = "%.2f" % (round((svm_hog_model.predict_proba([eye_hog2])[:,1])[0], 4) * 100)
                
    #display probabilities on the frame
    cv2.putText(frame, f'LEFT EYE: {eye_prob1}', (30, 450), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255))
    cv2.putText(frame, f'RIGHT EYE: {eye_prob2}', (230, 450), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255))
    
    #display status result on the frame
    cv2.putText(frame, f'STATUS: {tempstat}', (30, 90), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 0, 255))
            
    return tempstat

## 2.2 Function for extracting HOG features

In [3]:
def get_hog(image):
    #getting hog features for image
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return fd

## 2.3 Function for playing alert

In [31]:
def sound_alert(data, fs):
    #playing the sound file
    sd.play(data, fs)

# 3. Loading the required files, model

## 3.1 Reading the model file

In [5]:
#initialsing the file path for the model
model_file_name = 'model_SVM_HOG.sav'
model_path = f'{os.path.dirname(os.getcwd())}\\{model_file_name}'

#printing the model file, if found
print(f'The model file is located at: {model_path}')

The model file is located at: C:\Users\Sarthak\Desktop\Projects\model_SVM_HOG.sav


In [6]:
#reading the model file
with open(model_path, 'rb') as file:  
    svm_hog_model = pickle.load(file)

## 3.2 Haar cascade files

In [7]:
#Haar Cascade files for face and eye detection
face_cascade = cv2.CascadeClassifier('files\\haar\\haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('files\\haar\\haarcascade_eye.xml')

## 3.3 Sound file for alert

In [8]:
#location for sound file
alert_filename = 'files\\sound\\alert.wav'

# 4. Main process flow

## 4.1 Declaring variables

In [9]:
alarm_on = False #trigger for alert
frame_counter = 0 #counter of frames for drowsy state
status = '' #status of eyes

## 4.2 Check if the camera works

In [ ]:
#streaming cam feed from phone
#url, when on uni-wifi
#url = "http://10.108.50.88:4747/video" 
#url, when on mobile-hotspot
#url = "http://10.9.149.216:4747/video" 
#cap = cv2.VideoCapture(url)

In [38]:
#streaming live video from webcam
cap = cv2.VideoCapture(-1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError("ERROR: Unable to open webcam on device!")

## 4.3 Detecting if the driver is drowsy

In [39]:
#reading the sound file
data, fs = sf.read(alert_filename, dtype='float32')

while True:
    #reading video in frame
    ret, frame = cap.read()
    
    #rotating camera, when using mobile camera
    #frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    
    #grayscaling the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #checking drowsy status
    status = detect_drowsy(gray, frame, status)
    
    #increment counter if eyes are closed
    if status == 'EYES CLOSED':
        frame_counter += 1
        #if counter more than 50 or 4 secs
        if frame_counter >= 50:
            #switch the alarm
            alarm_on = True
            if alarm_on == True:
                #display alert
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)
                #ring the alarm corresponding to the alert
                t = Thread(target=sound_alert(data, fs))
                t.daemon = True
                t.start()
                
    #if the driver is awake
    elif status == 'EYES OPEN':
        #reset the alarm and frame counter
        frame_counter = 0
        alarm_on = False    
    
    cv2.imshow('Drowsiness Detector', frame)
    #exit on pressing 'q'
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()